## Outbox Pattern
This pattern is useful when we want to write to a database and publish message to a queue as one transaction. Either both operations succeed or neither do. As an example, lets say we have an *Order Service* that needs to do two things simultaneously:
- Update its local database by setting order status as "SHIPPED"`
- Notify other services (e.g., the Billing Service or Notification Service) by publishing a message ("Order Shipped Event").

Possible failure scenarios:
- if the database was updated, but publishing message failed then the other services never know the order shipped leading to data loss
- if we publish the message first but fail to write to the database, then the other services act on incorrect information leading to data inconsistency

We can introduce *retry* on failure, but what if the hardware crashed? We would not be able to retry or even perform the second action.

**Solution:** involves introducing a second table called *outbox* table that records all the message publish actions that need to be performed. Then we combine writing to database and the outbox table in one transaction:

In [ ]:
%%sql
BEGIN TRANSACTION;
    UPDATE orders SET status = 'SHIPPED' WHERE id = 101;
    INSERT INTO outbox (topic, payload, sent) VALUES ('shipping_events', '{order_id: 101, status: SHIPPED}', false);
COMMIT;

Now writes to both tables succeeds or both fail.

We also need another service called *relay* service which polls the outbox table for new entries and publishes message to the appropriate message queue. Once the message has been published, update the outbox table entry:

In [ ]:
%%sql
UPDATE outbox SET sent = true WHERE...

If the database crashes after the `UPDATE` orders but before the `COMMIT`, neither the order status nor the outbox entry is recorded. If the database crashes after the `COMMIT`, the data is safe, and the relay service will ensure the message is eventually delivered to the queue.

<img src="images/outbox_pattern.png" />

What if we fail to update outbox table once we have published message to the queue? There is a possibility of publishing the message again. Outbox pattern solves the atomicity problem at the source but introduces the risk of *at-least-once* delivery duplication.

## Duplicate Messages
Message brokers look for an acknowledgment from the consumer to mark message as being processed successfully. If the acknowledgement is not received, the same message can be sent again. This leads to the problem of duplicate message being sent. Infact message brokers like Kafka guarantee deliver *at least once* semantics.

Processing same message twice can be problematic in certain scenarios.

**Solution:** there are two possible ways to handle duplicate messages:
- Having *idempotent consumer*. This may not always be possible.
- Tracking message: every message is marked with a unique identifier before being pushed to the queue. The consumer gets message from the queue and it checks its local database to verify if the message was previously received and processed. If it was, the message is simply discarded. If it was not, the message is consumed and the message status is set as processed or done.

<img src="images/message_tracking.png" />